In [30]:
import sqlite3
import pandas as pd
import pyodbc

In [31]:
conn1 : sqlite3.Connection = sqlite3.connect('database.sqlite3')
conn2 : pyodbc.Connection = pyodbc.connect(
	"Driver={ODBC Driver 17 for SQL Server};"
	"Server=srvdb04.seac.local;"
	"Trusted_Connection=yes;"
)

In [70]:
def get_create(conn, table_name, db=None, dbtype='sqlite'):
	if dbtype.lower() == 'sqlite':
		query = f""" SELECT sql FROM sqlite_master WHERE type = 'table' AND name != 'sqlite_sequence' AND name = ? """
		df = pd.read_sql(query, conn, params=(table_name,))
		return df['sql'][0]
	elif dbtype.lower() == 'tsql':
		query = f"""SELECT * FROM {f'[{db}].' if db is not None else '' }INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = ? """
		df = pd.read_sql(query, conn, params=(table_name,))
		create = f"""CREATE TABLE {table_name} ("""
		for x in range(len(df)):
			line = df.iloc[x]
			create += f"""\n\t{line['COLUMN_NAME']} {line['DATA_TYPE']},"""
		create = f"""{create[0:len(create)-1]}\n);"""
		return create
	raise Exception('dbtype not valid')

In [77]:
get_create(conn=conn1, table_name='Products', dbtype='sqlite')

"CREATE TABLE [Products](\n   [ProductID]INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,\n   [ProductName]TEXT NOT NULL,\n   [SupplierID]INTEGER,\n   [CategoryID]INTEGER,\n   [QuantityPerUnit]TEXT,\n   [UnitPrice]NUMERIC DEFAULT 0,\n   [UnitsInStock]INTEGER DEFAULT 0,\n   [UnitsOnOrder]INTEGER DEFAULT 0,\n   [ReorderLevel]INTEGER DEFAULT 0,\n   [Discontinued]TEXT NOT NULL DEFAULT '0',\n    CHECK ([UnitPrice]>=(0)),\n    CHECK ([ReorderLevel]>=(0)),\n    CHECK ([UnitsInStock]>=(0)),\n    CHECK ([UnitsOnOrder]>=(0)),\n\tFOREIGN KEY ([CategoryID]) REFERENCES [Categories] ([CategoryID]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\n\tFOREIGN KEY ([SupplierID]) REFERENCES [Suppliers] ([SupplierID]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)"

In [76]:
get_create(conn=conn2, table_name='DIM_HD_Chiamate', db='DwBusiness', dbtype='tsql')

C:\Users\vttndr\AppData\Local\Temp\ipykernel_13108\4175468803.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=(table_name,))


'CREATE TABLE DIM_HD_Chiamate (\n\tIdDatabase int,\n\tIdChiamata bigint,\n\tIdRiga int,\n\tIdCliente bigint,\n\tData varchar,\n\tStato char,\n\tUrgenza char,\n\tSollecito char,\n\tDurata int,\n\tLogin varchar,\n\tGruppo varchar,\n\tCausa varchar,\n\tUpdateDataOra datetime\n);'